In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


In [3]:
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.25, 0.25, 0.25])

data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

In [4]:
# import os
# import random
# import torch
# from torchvision import datasets, transforms
# from torch.utils.data import Subset
# data_dir = "D:/P2023/DATA/glomer_cg/"

# # selected_subfolders = ["MGN", "IGA"]

# # Create a single merged dataset
# merged_dataset = datasets.ImageFolder(data_dir, data_transforms['train'])
# # idx = [i for i in range(len(merged_dataset)) if merged_dataset.imgs[i][1] == merged_dataset.class_to_idx['MGN'] or merged_dataset.imgs[i][1] == merged_dataset.class_to_idx['IGA']]
# # merged_dataset = Subset(merged_dataset, idx)
# # print(merged_dataset.imgs[10][1])
# # print(subset)
# # Shuffle the merged dataset randomly
# random.seed(42)  # You can choose any random seed for reproducibility
# indices = list(range(len(merged_dataset)))
# random.shuffle(indices)

# # Define the split ratio (e.g., 80% for training and 20% for validation)
# split_ratio = 0.8  # You can adjust this ratio as needed

# # Calculate the split indices
# split_index = int(len(indices) * split_ratio)
# train_indices = indices[:split_index]
# val_indices = indices[split_index:]

# # Create data loaders for training and validation using the split indices
# train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
# val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

# train_loader = torch.utils.data.DataLoader(merged_dataset, batch_size=4, sampler=train_sampler, num_workers=0)
# val_loader = torch.utils.data.DataLoader(merged_dataset, batch_size=4, sampler=val_sampler, num_workers=0)

# # Example usage:
# print(f"Number of training samples: {len(train_indices)}")
# print(f"Number of validation samples: {len(val_indices)}")


Number of training samples: 10171
Number of validation samples: 2543


In [5]:
import os
import random
import torch
from torchvision import datasets, transforms

data_dir = "D:/P2023/DATA/glomer_cg"

# Create a single merged dataset
merged_dataset = datasets.ImageFolder(data_dir, data_transforms['train'])

# Shuffle the merged dataset randomly
random.seed(42)  # You can choose any random seed for reproducibility
indices = list(range(len(merged_dataset)))
random.shuffle(indices)

# Define the split ratio (e.g., 80% for training and 20% for validation)
split_ratio = 0.8  # You can adjust this ratio as needed

# Calculate the split indices
split_index = int(len(indices) * split_ratio)
train_indices = indices[:split_index]
val_indices = indices[split_index:]

# Create data loaders for training and validation using the split indices
train_sampler = torch.utils.data.SubsetRandomSampler(train_indices)
val_sampler = torch.utils.data.SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(merged_dataset, batch_size=4, sampler=train_sampler, num_workers=0)
val_loader = torch.utils.data.DataLoader(merged_dataset, batch_size=4, sampler=val_sampler, num_workers=0)

# Example usage:
print(f"Number of training samples: {len(train_indices)}")
print(f"Number of validation samples: {len(val_indices)}")


Number of training samples: 10171
Number of validation samples: 2543


In [8]:

# num_datapoint = len(raw_datasets)

image_datasets = {
    "train": train_loader.dataset,
    "val": val_loader.dataset
}
dataloaders = {
    x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=0)
    for x in ['train', 'val']
}

In [9]:
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(class_names)
print(device)

['IGA', 'MGN']
cuda:0


In [10]:
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()


# Get a batch of training data
inputs, classes = next(iter(dataloaders['train']))

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [11]:
print(classes)

tensor([1, 0, 0, 1])


In [18]:
# complete
from datetime import datetime
log_path ="D:/P2023/LOG/resnet152_class2.txt"
# print and fprint at the same time
def pprint(output = '\n' , filename = log_path, show_time = False):
    print(output)
    with open(filename, 'a') as f:
        if show_time:
            f.write(datetime.now().strftime("[%Y-%m-%d %H:%M:%S] "))

        f.write(str(output))
        f.write('\n')
pprint("test", show_time=True)

test


In [19]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    
    best_recallmn = 0.0 # ***
    best_recallig = 0.0 # ***
    
    best_precmn = 0.0 # ***
    best_precig = 0.0 # ***
    
    best_f1mn = 0.0 # ***
    best_f1ig = 0.0 # ***

    for epoch in range(num_epochs):
        pprint('Epoch {}/{}'.format(epoch, num_epochs - 1), show_time=True)
        pprint('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            
            tp_positive = 0
            fp_positive = 0
            tn_negative = 0
            fn_negative = 0
            # positive_other = 0
            # negative_other = 0
            
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)


                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                
                # MN -> positive
                # iga -> negative
                tp_positive += torch.sum((preds == 1) & (labels.data == 1))
                fp_positive += torch.sum((preds == 1) & (labels.data == 0))
                tn_negative += torch.sum((preds == 0) & (labels.data == 0))
                fn_negative += torch.sum((preds == 0) & (labels.data == 1))
                # positive_other += torch.sum((preds == 2) & (labels.data == 1))
                # negative_other += torch.sum((preds == 2) & (labels.data == 0))
                
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            
            epoch_recallmn = tp_positive.double() / (tp_positive + fn_negative) # ***
            epoch_recallig = tn_negative.double() / (tn_negative + fp_positive) # ***
            
            epoch_precmn = tp_positive.double() / (tp_positive + fp_positive) # ***
            epoch_precig = tn_negative.double() / (tn_negative + fn_negative) # ***
            
            epoch_f1mn = (2 * epoch_recallmn * epoch_precmn) / (epoch_recallmn + epoch_precmn) # ***
            epoch_f1ig = (2 * epoch_recallig * epoch_precig) / (epoch_recallig + epoch_precig) # ***
            
            # print('{} Loss: {:.4f} Acc: {:.4f} Recall_MGN: {:.4f} Recall_IGAN: {:.4f} Precision_MGN: {:.4f} Precision_IGAN: {:.4f} F1_MGN: {:.4f} F1_IGAN: {:.4f}'.format(
            #     phase, epoch_loss, epoch_acc, epoch_recallmn, epoch_recallig, epoch_precmn, epoch_precig, epoch_f1mn, epoch_f1ig)) # ***
            pprint('{} Loss: {:.4f} Accuracy: {:.4f} \n     Recall Precision F1_score\n MGN: {:.4f} {:.4f} {:.4f} \n IGAN:{:.4f} {:.4f} {:.4f} \n'.format(
                    phase, epoch_loss, epoch_acc, epoch_recallmn, epoch_precmn, epoch_f1mn, epoch_recallig, epoch_precig, epoch_f1ig)) # ***

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                
            if phase == 'val' and epoch_recallmn > best_recallmn: # ***
                best_recallmn = epoch_recallmn
            if phase == 'val' and epoch_recallig > best_recallig: # ***
                best_recallig = epoch_recallig  
                
            if phase == 'val' and epoch_precmn > best_precmn: # ***
                best_precmn = epoch_precmn
            if phase == 'val' and epoch_precig > best_precig: # ***
                best_precig = epoch_precig    
                
            if phase == 'val' and epoch_f1mn > best_f1mn: # ***
                best_f1mn = epoch_f1mn
            if phase == 'val' and epoch_f1ig > best_f1ig: # ***
                best_f1ig = epoch_f1ig   
                
        print()

    time_elapsed = time.time() - since
    pprint('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    pprint('Best val Acc: {:.4f} \n     Recall Precision F1_score\n MGN: {:.4f} {:.4f} {:.4f}\n IGAN:{:.4f} {:.4f} {:.4f}\n'.format(
                best_acc, best_recallmn, best_precmn, best_f1mn, best_recallig, best_precig, best_f1ig)) # ***

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [20]:
def train_mod(true_None, num_epoch):
    model = models.resnet152(weights=true_None)
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.001)
    step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)
    model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs=num_epoch)    
    return model

In [ ]:
weight_name = "D:/P2023/WEIGHT/res152_50t_class2.pt"

pprint(f"Model at {weight_name}")
model = train_mod(True, 50)
torch.save(model.state_dict(), weight_name)

Model at D:/P2023/WEIGHT/res152_50t_class2.pt
Epoch 0/49
----------
train Loss: 0.5495 Accuracy: 0.7139 
     Recall Precision F1_score
 MGN: 0.5566 0.6615 0.6046 
 IGAN:0.8156 0.7397 0.7758 

val Loss: 0.4108 Accuracy: 0.8200 
     Recall Precision F1_score
 MGN: 0.6673 0.8417 0.7444 
 IGAN:0.9188 0.8101 0.8610 


Epoch 1/49
----------
train Loss: 0.4778 Accuracy: 0.7703 
     Recall Precision F1_score
 MGN: 0.6661 0.7265 0.6950 
 IGAN:0.8377 0.7949 0.8157 

val Loss: 0.3263 Accuracy: 0.8628 
     Recall Precision F1_score
 MGN: 0.7738 0.8629 0.8160 
 IGAN:0.9204 0.8628 0.8907 


Epoch 2/49
----------
train Loss: 0.4412 Accuracy: 0.7971 
     Recall Precision F1_score
 MGN: 0.7130 0.7566 0.7341 
 IGAN:0.8515 0.8209 0.8359 

val Loss: 0.3223 Accuracy: 0.8548 
     Recall Precision F1_score
 MGN: 0.8449 0.7976 0.8206 
 IGAN:0.8612 0.8956 0.8781 


Epoch 3/49
----------
train Loss: 0.4035 Accuracy: 0.8165 
     Recall Precision F1_score
 MGN: 0.7448 0.7786 0.7613 
 IGAN:0.8629 0.8393 0.8

In [ ]:
weight_name = "D:/P2023/WEIGHT/res152_50n_class2.pt"

pprint(f"Model at {weight_name}")
model = train_mod(None, 50)
torch.save(model.state_dict(), weight_name)

In [ ]:
# model = train_mod(None, 50)
# torch.save(model.state_dict(), "D:/P2023/WEIGHT/res18_50n.pt")

In [ ]:
# model = train_mod(True, 50)
# torch.save(model.state_dict(), "D:/P2023/WEIGHT/res18_50t.pt")